# Train pam2vec

In [1]:
# Load packages
import os
import pandas as pd
import numpy as np
import yaml
import nltk
import re
from gensim import models
from stop_words import get_stop_words
from scipy.spatial import distance


In [2]:
with open('config_file.yaml', 'r') as f:
    config = yaml.load(f)

In [3]:
# Read the dataset
df = pd.read_csv(config['ROOT_PATH'] + '/data/' + 'proposals_clean.csv', sep=',', encoding='utf-8')

In [4]:
df.head()

,Unnamed: 0,proposal,origin,scope,district,category,subcategory,author,author_name,created_at,...,total_neutral_comments,total_negative_comments,rejected_message,language,action,proposal_list,title_ca_action,description_ca_action,title_es_action,description_es_action
0,0,3591.0,citizenship,district,Nou Barris,Economia plural,Un nou lideratge públic,5187,VILLARRASA,2/27/2016,...,0.0,0.0,No hi ha competències i l’Ajuntament no dispos...,es,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2747.0,organization,district,Gràcia,Transició ecològica,Medi ambient i espai públic,6285,NaN,2/17/2016,...,0.0,0.0,No podem comprometre’ns a desenvolupar aquesta...,ca,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3158.0,citizenship,district,Horta - Guinardó,Bon viure,Educació i coneixement,512,Solidaritat Catalana per la Independència,2/19/2016,...,3.0,0.0,None,ca,5061.0,"['360', '1648', '3158', '6161']",Analitzar la necessitat i la idoneïtat de l'IE...,Estudiar les necessitats i la situació real pe...,Analizar la necesidad y la idoneidad del IES e...,Estudiar las necesidades y la situación real p...
3,3,8968.0,citizenship,city,NaN,Transició ecològica,Mobilitat sostenible,15512,Archie,4/6/2016,...,0.0,0.0,No podem comprometre’ns a desenvolupar aquesta...,ca,NaN,NaN,NaN,NaN,NaN,NaN
4,4,6774.0,citizenship,district,Gràcia,Transició ecològica,Urbanisme per als barris,15153,Xavier Sisternas,3/24/2016,...,0.0,0.0,None,ca,2717.0,"['2391', '2407', '2415', '2418', '2420', '2421...",Pla de millora integral i manteniment de l’esp...,"Identificar, prioritzar i dur a terme actuacio...",Plan de mejora integral y mantenimiento del es...,"Identificar, priorizar y llevar a cabo actuaci..."


In [5]:
len(df)

11448

In [6]:
df.columns

Index(['Unnamed: 0', 'proposal', 'origin', 'scope', 'district', 'category',
       'subcategory', 'author', 'author_name', 'created_at', 'votes',
       'comments', 'url', 'status', 'title_es', 'title_ca', 'description_es',
       'description_ca', 'group_author', 'total_positive_comments',
       'total_neutral_comments', 'total_negative_comments', 'rejected_message',
       'language', 'action', 'proposal_list', 'title_ca_action',
       'description_ca_action', 'title_es_action', 'description_es_action'],
      dtype='object')

In [7]:
# Create series with texts
text_ca = df.title_ca.drop_duplicates().astype(str) + '. ' + df.description_ca.drop_duplicates().astype(str)  
text_ca_actions = df.title_ca_action.drop_duplicates().astype(str) + '. ' + df.description_ca_action.drop_duplicates().astype(str)
text_ca = text_ca.append(text_ca_actions, ignore_index=True)
text_ca = text_ca.dropna()

text_es = df.title_es.drop_duplicates().astype(str) + '. ' + df.description_es.drop_duplicates().astype(str)  
text_es_actions = df.title_es_action.drop_duplicates().astype(str) + '. ' + df.description_es_action.drop_duplicates().astype(str)
text_es = text_es.append(text_es_actions, ignore_index=True)
text_es = text_es.dropna()

In [8]:
# Functions to delete stopwords and tokenize
def tokenize_ca(text):
    stop_words_ca = get_stop_words('catalan')
    stop_words_ca = stop_words_ca + ['a', 'al', 'als', 'del', 'dels', 'et', 'la', 'que', 'més', 'no', 'nan']

    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens = [token.replace(" l '", "l'") for token in tokens]
    tokens = [token.replace("l'", "") for token in tokens]
    tokens = [token.replace("d'", "") for token in tokens]
    tokens = [token.replace("m'", "") for token in tokens]
    tokens = [token.replace("s'", "") for token in tokens]
    tokens = [token.replace(".", "") for token in tokens]
    tokens = [token.replace("-ne", "") for token in tokens]
    tokens = [token.replace("-se", "") for token in tokens]
    tokens = [token.replace("'", "") for token in tokens]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            if token not in stop_words_ca:
                filtered_tokens.append(token)
                
    return filtered_tokens

def tokenize_es(text):
    stop_words_es = get_stop_words('spanish')
    stop_words_es = stop_words_es + ['nan']
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens = [token.replace("'", "") for token in tokens]
    tokens = [token.replace(".", "") for token in tokens]
    tokens = [token.replace(",", "") for token in tokens]
    tokens = [token.replace(";", "") for token in tokens]
    tokens = [token.replace(":", "") for token in tokens]
    tokens = [token.replace("?", "") for token in tokens]
    tokens = [token.replace("!", "") for token in tokens]
    tokens = [token.replace("¡", "") for token in tokens]
    tokens = [token.replace("¿", "") for token in tokens]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            if token not in stop_words_es:
                filtered_tokens.append(token)
 
    return filtered_tokens


In [9]:
# Tokenize text
text_ca = text_ca.apply(tokenize_ca)
text_es = text_es.apply(tokenize_es)

In [10]:
len(text_ca)

11660

In [11]:
# Train model: word2vec
vector_size = 100
window_size = 4
min_count = 2
workers_num = 10

model_ca = models.Word2Vec(size = vector_size, window = window_size, min_count = min_count, workers = workers_num, sg = 1)
model_ca.build_vocab(text_ca)  # prepare the model vocabulary
model_ca.train(text_ca, total_examples=model_ca.corpus_count, epochs=500)  # train word vectors



model_es = models.Word2Vec(size = vector_size, window = window_size, min_count = min_count, workers = workers_num, sg = 1)
model_es.build_vocab(text_es)  # prepare the model vocabulary
model_es.train(text_es, total_examples=model_es.corpus_count, epochs=500)  # train word vectors

(130242950, 139296000)

In [12]:
model_ca.wv.similarity('bicicleta', 'ciutat')



0.102847874

In [13]:
model_es.wv.similarity('bicicleta', 'ciudad')

0.11721142

In [14]:
#Functions to compute pam2vec 
def normalize(word_vec):
    norm = np.linalg.norm(word_vec)
    if norm == 0: 
        return word_vec
    return word_vec/norm


def pam2vec(text, lang):
    """text and language and returns the mean of all word2vec of the text"""
    words_list = text
    if lang == 'ca':
        model = model_ca
        words_list = [word for word in words_list if word in model.wv.vocab]
    else:
        model = model_es
        words_list = [word for word in words_list if word in model.wv.vocab]

    if len(words_list) > 0:
        return np.mean(normalize(model[words_list]), axis=0)
    else:
        return np.nan * np.empty(vector_size)

In [20]:
# Jaccard similarity function
def jaccard_distance(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    jacc_sim = len(s1.intersection(s2)) / len(s1.union(s2))
    return 1-jacc_sim

In [16]:
# Compute pam2vec
pam2vec_proposals_ca = []
pam2vec_actions_ca = []
pam2vec_proposals_es = []
pam2vec_actions_es = []

for index, row in df.iterrows():
    proposal_text = str(row['title_ca']) + '. ' + str(row['description_ca'])  
    action_text = str(row['title_ca_action']) + '. ' + str(row['description_ca_action']) 
    proposal_text = tokenize_ca(proposal_text)
    action_text = tokenize_ca(action_text)

    pam2vec_proposals_ca.append(pam2vec(proposal_text, 'ca'))
    pam2vec_actions_ca.append(pam2vec(action_text, 'ca'))
    
    proposal_text = str(row['title_es']) + '. ' + str(row['description_es'])  
    action_text = str(row['title_es_action']) + '. ' + str(row['description_es_action']) 
    proposal_text = tokenize_es(proposal_text)
    action_text = tokenize_es(action_text)

    pam2vec_proposals_es.append(pam2vec(proposal_text, 'es'))
    pam2vec_actions_es.append(pam2vec(action_text, 'es'))

/home/avaldivia/env35/lib/python3.5/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [22]:
# Compute distances between vectors
for index, row in df.iterrows():
    v1 = pam2vec_proposals_ca[index]
    v2 = pam2vec_actions_ca[index]
    df.loc[index, 'cos_dist_ca'] = round(distance.cosine(v1, v2), 3)
    df.loc[index, 'eucl_dist_ca'] = round(distance.sqeuclidean(v1, v2), 3)
    df.loc[index, 'jacc_dist_ca'] = round(jaccard_distance(v1, v2), 3)
    df.loc[index, 'manh_dist_ca'] = round(distance.cityblock(v1, v2), 3)

    v1 = pam2vec_proposals_es[index]
    v2 = pam2vec_actions_es[index]
    df.loc[index, 'cos_dist_es'] = round(distance.cosine(v1, v2), 3)
    df.loc[index, 'eucl_dist_es'] = round(distance.sqeuclidean(v1, v2), 3)
    df.loc[index, 'jacc_dist_es'] = round(jaccard_distance(v1, v2), 3)
    df.loc[index, 'manh_dist_es'] = round(distance.cityblock(v1, v2), 3)    

In [23]:
df.head()

,Unnamed: 0,proposal,origin,scope,district,category,subcategory,author,author_name,created_at,...,eucl_dist_ca,manh_dist_ca,cos_sim_es,jacc_sim_es,eucl_dist_es,manh_dist_es,cos_dist_ca,jacc_dist_ca,cos_dist_es,jacc_dist_es
0,0,3591.0,citizenship,district,Nou Barris,Economia plural,Un nou lideratge públic,5187,VILLARRASA,2/27/2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
1,1,2747.0,organization,district,Gràcia,Transició ecològica,Medi ambient i espai públic,6285,NaN,2/17/2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
2,2,3158.0,citizenship,district,Horta - Guinardó,Bon viure,Educació i coneixement,512,Solidaritat Catalana per la Independència,2/19/2016,...,0.012,0.879,0.333,0.014,0.014,0.908,0.32,1.0,0.333,1.0
3,3,8968.0,citizenship,city,NaN,Transició ecològica,Mobilitat sostenible,15512,Archie,4/6/2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
4,4,6774.0,citizenship,district,Gràcia,Transició ecològica,Urbanisme per als barris,15153,Xavier Sisternas,3/24/2016,...,0.007,0.651,0.479,0.008,0.008,0.703,0.46,1.0,0.479,1.0


In [24]:
# Write dataset
df.to_csv(config['ROOT_PATH'] + '/data/' + 'proposals_dists.csv', sep=',', encoding='utf-8')